# Machine Learning in Finance - Data Processing

This notebook will focus on processing the data with ML techniques (RandomForest), before applying ML techniques to find the top characteristics for price prediction.

***Note: This part used a package `missingpy` that requires old versions of scipy. To avoid unnecessary conflict, please uncomment the first cell to create a conda environment just for this pre-processing task. Do not forget to set the created environment as you IDE interpreter. Finally, `missingpy` has a second model which is deprecated and cause issues while importing. Then please follow the step of the video [here](https://www.youtube.com/watch?v=_886JGYt1Ts).***

*Authors:* [Mina Attia](https://people.epfl.ch/mina.attia), [Arnaud Felber](https://people.epfl.ch/arnaud.felber), [Milos Novakovic](https://people.epfl.ch/milos.novakovic), [Rami Atassi](https://people.epfl.ch/rami.atassi) & [Paulo Ribeiro](https://people.epfl.ch/paulo.ribeirodecarvalho)

In [1]:
#!conda create --name impute python=3.8
#!conda activate impute
#!/opt/anaconda3/envs/impute/bin/pip install missingpy
#!/opt/anaconda3/envs/impute/bin/pip install scikit_learn
#!/opt/anaconda3/envs/impute/bin/pip install pandas

## Import

In [2]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from helpers import load_data_df
from data_processing import impute_data

import warnings

# Filter out UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)

%load_ext autoreload
%autoreload 2

## Data

Load the dataset.

In [3]:
file_path = 'data/data_reduce.csv'

data = load_data_df(file_path=file_path)

## Impute

Retrieve the missing values using Machine Learning techniques. To do so, we use the API from `missingpy` and call the MissForest algorithm. MissForest imputes missing values using Random Forests in an iterative fashion.

In [4]:
chunks = 10
data_chunks = np.array_split(data, chunks)

imputed_data_list = []
for data_chunk in tqdm(data_chunks[1:], total=chunks):
    imputed_data = impute_data(data=data_chunk.reset_index(drop=True),
                               seed=42)
    imputed_data_list.append(imputed_data)
    
imputed_data_full = pd.concat(imputed_data_list, ignore_index=True).drop(columns='Unnamed: 0')
    
# Check if there are any NaN values in the DataFrame
if imputed_data_full.isna().any().any():
    print("There still has NaN values in the DataFrame.")
else:
    print("There are no NaN values in the DataFrame.")

## Data Processing

Since no more Nan values are in price, we can compute the return of each record. To do so we compute the diff between every related record and divide by the current price. This way each record is associate to the return of the asset the next month. 

In [5]:
# Rename correctly the column
imputed_data_full.rename(columns={'Price': 'log_price'}, inplace=True)

# Position the log_price column at the end of the dataframe
column_to_move = imputed_data_full.pop('log_price')
imputed_data_full['log_price'] = column_to_move

# Create the column price by takin the exponential of the log price
imputed_data_full['price'] = np.exp(-1*imputed_data_full['log_price'])

# Compute the log return and simple return
# Sort the DataFrame by date
imputed_data_full.sort_values(by=['permno', 'date'], inplace=True)

# Calculate logarithmic returns for each asset separately
imputed_data_full['log_diff'] = imputed_data_full.groupby('permno')['log_price'].diff()
imputed_data_full['diff'] = imputed_data_full.groupby('permno')['price'].diff()

# Shift the returns by one month to align with the next month's data
imputed_data_full['log_diff'] = imputed_data_full.groupby('permno')['log_diff'].shift(-1)
imputed_data_full['diff'] = imputed_data_full.groupby('permno')['diff'].shift(-1)

# Drop the last row for each asset since it will have NaN due to shifting
imputed_data_full = imputed_data_full.dropna()

# Compute the return ratio now
imputed_data_full['log_return'] = imputed_data_full['log_diff'] / imputed_data_full['log_price']
imputed_data_full['return'] = imputed_data_full['diff'] / imputed_data_full['price']

# Reset index if needed
imputed_data_full.reset_index(drop=True, inplace=True)
imputed_data_full.drop(columns=['diff', 'log_diff'], inplace=True)

## Store Impute Data

In [6]:
imputed_data_full.to_csv('data/imputed/data_imputed.csv', index=False)